# function stores

In [1]:
def _classification_report(prediction , nameOfModel) : 
    #PREPARE SECCTION
    target_names = df_label["sleep"].unique().sort()
    clf_rbf_raw = classification_report(y_test,prediction ,target_names=target_names  ,output_dict=True)
    clf_rbf = pd.DataFrame(clf_rbf_raw)
    
    #PLOTING SECCTION
    fig = px.imshow(clf_rbf.iloc[:-1, :].T, text_auto=True ,aspect="auto",color_continuous_scale='GnBu'  )
    fig.update_layout(height=400, width=500, title_text='Classification Report of '+str(nameOfModel) +"SVM")
    fig.show()

# import lib

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler ,OneHotEncoder
from sklearn.model_selection import  GridSearchCV ,RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
from scipy import stats

# (a) Load and Prepare Data

In [3]:
# Load data from csv 3 files
# acceleration.txt, heartrate.txt, labeled_sleep.txt
acceleration = "./Resources/46343_acceleration.txt"
heartrate = "./Resources/46343_heartrate.txt"
labeled_sleep="./Resources/46343_labeled_sleep.txt"
init_acc = pd.read_csv(acceleration, sep = ' ',names=['timedelta', 'accX', 'accY', 'accZ'])
init_heartrate = pd.read_csv(heartrate, sep = ',',names=['timedelta', 'heartrate'])
init_sleep = pd.read_csv(labeled_sleep, sep = ' ',names=['timedelta', 'sleep'])

acceleration2 = "./Resources/844359_acceleration.txt"
heartrate2 = "./Resources/844359_heartrate.txt"
labeled_sleep2="./Resources/844359_labeled_sleep.txt"
init_acc2 = pd.read_csv(acceleration2, sep = ' ',names=['timedelta', 'accX', 'accY', 'accZ'])
init_heartrate2 = pd.read_csv(heartrate2, sep = ',',names=['timedelta', 'heartrate'])
init_sleep2 = pd.read_csv(labeled_sleep2, sep = ' ',names=['timedelta', 'sleep'])

In [4]:
init_acc2.head()

,timedelta,accX,accY,accZ
0,-500.492604,-0.023804,-0.535217,-0.832169
1,-500.474720,-0.015472,-0.535690,-0.833450
2,-500.455065,-0.019379,-0.528381,-0.834518
3,-500.434675,-0.013473,-0.531311,-0.836334
4,-500.414981,-0.015366,-0.533752,-0.839294


In [5]:
init_heartrate2.head()

,timedelta,heartrate
0,-604720.29398,68
1,-604438.91905,64
2,-604032.54366,63
3,-603757.54405,64
4,-603454.66864,64


In [6]:
init_sleep2.head()

,timedelta,sleep
0,0,-1
1,30,-1
2,60,-1
3,90,-1
4,120,-1


In [7]:
acc_max_date = init_acc2["timedelta"].max()
acc_min_date = init_acc2["timedelta"].min()
print("acceleration start :" ,acc_min_date)
print("acceleration end :" ,acc_max_date)

acceleration start : -500.492604017
acceleration end : 28828.1484408


In [8]:
sleep_min = init_sleep2["timedelta"].min()
sleep_max = init_sleep2["timedelta"].max()
print("sleep label start :" ,sleep_min)
print("sleep label end :" ,sleep_max)

sleep label start : 0
sleep label end : 28440


In [9]:
hrate_min = init_heartrate2["timedelta"].min()
hrate_max = init_heartrate2["timedelta"].max()
print("heart rate start :" ,hrate_min)
print("heart rate end :" ,hrate_max)

heart rate start : -604720.29398
heart rate end : 28993.7883699


### select only intersected timedelta (ACC, HeartR, SleepL) (ช่วงเวลาที่มีข้อมูลใกล้กัน)

In [92]:
seleted_min  = 100
seleted_max = 5000

p2min = 7000
p2max = 12000

In [93]:
acc_new = init_acc[(init_acc['timedelta'] > seleted_min ) & (init_acc['timedelta'] <= seleted_max)]
heartRate_new = init_heartrate[(init_heartrate['timedelta'] >= seleted_min ) & (init_heartrate['timedelta'] < seleted_max)]
sleepLabel_new = init_sleep[(init_sleep['timedelta'] >= seleted_min ) &(init_sleep['timedelta'] <= seleted_max)]

In [94]:
p2_acc = init_acc[(init_acc2['timedelta'] > p2min ) & (init_acc2['timedelta'] <= p2max)]
p2_hr = init_heartrate[(init_heartrate2['timedelta'] >= p2min ) & (init_heartrate2['timedelta'] < p2max)]
p2_sleep = init_sleep[(init_sleep2['timedelta'] >= p2min ) &(init_sleep2['timedelta'] <= p2max)]

# Convert to datetime and round to second,

In [95]:
acc_new["timedelta"] = pd.DataFrame(pd.to_timedelta(acc_new['timedelta'], unit='s').round('s'))
p2_acc["timedelta"] = pd.DataFrame(pd.to_timedelta(p2_acc['timedelta'], unit='s').round('s'))

# Average rounding duplicated time

In [96]:
df_acc_X = acc_new.groupby('timedelta')['accX'].mean().reset_index()
df_acc_Y = acc_new.groupby('timedelta')['accY'].mean().reset_index()
df_acc_Z = acc_new.groupby('timedelta')['accZ'].mean().reset_index()

p2x = p2_acc.groupby('timedelta')['accX'].mean().reset_index()
p2y = p2_acc.groupby('timedelta')['accY'].mean().reset_index()
p2z = p2_acc.groupby('timedelta')['accZ'].mean().reset_index()

In [97]:
acc_df = df_acc_X.set_index('timedelta').join(df_acc_Y.set_index('timedelta')).join(df_acc_Z.set_index('timedelta'))
p2_acc_df = p2x.set_index('timedelta').join(p2y.set_index('timedelta')).join(p2z.set_index('timedelta'))

# (c) Load and Prepare Data (Heart rate)

In [98]:
heartRate_new["timedelta"] = pd.DataFrame(pd.to_timedelta(heartRate_new['timedelta'], unit='s').round('1s'))
p2_hr["timedelta"] = pd.DataFrame(pd.to_timedelta(p2_hr['timedelta'], unit='s').round('1s'))

In [99]:
# Group duplicate time
heartRate_new = heartRate_new.groupby('timedelta')['heartrate'].mean().reset_index()
p2_hr = p2_hr.groupby('timedelta')['heartrate'].mean().reset_index()

In [100]:
resample_rule = '1s'
heartrate_df = heartRate_new.set_index('timedelta').resample(resample_rule,).median().ffill()
p2_hr_df = p2_hr.set_index('timedelta').resample(resample_rule,).median().ffill()

# (c) Load and Prepare Data (Sleep Label)

In [101]:
sleepLabel_new["timedelta"] = pd.DataFrame(pd.to_timedelta(sleepLabel_new['timedelta'], unit='s').round('1s'))
p2_sleep["timedelta"] = pd.DataFrame(pd.to_timedelta(p2_sleep['timedelta'], unit='s').round('1s'))


In [102]:
sleeplabel_df = sleepLabel_new.set_index('timedelta').resample(resample_rule,).median().ffill()
p2_sleep_df = p2_sleep.set_index('timedelta').resample(resample_rule,).median().ffill()
p2_sleep_df

,sleep
timedelta,
0 days 01:57:00,2.0
0 days 01:57:01,2.0
0 days 01:57:02,2.0
0 days 01:57:03,2.0
0 days 01:57:04,2.0
...,...
0 days 03:19:56,1.0
0 days 03:19:57,1.0
0 days 03:19:58,1.0


# ( e) Merge Data and Standardized data

In [103]:
df2 = []
df2= pd.merge_asof(p2_acc_df, p2_hr_df, on='timedelta')
df2 = pd.merge_asof(df2, p2_sleep_df, on='timedelta')
df2

,timedelta,accX,accY,accZ,heartrate,sleep
0,0 days 01:30:34,0.469599,-0.147249,-0.871480,NaN,NaN
1,0 days 01:30:35,0.475812,-0.145139,-0.868174,NaN,NaN
2,0 days 01:30:36,0.472598,-0.146138,-0.870019,NaN,NaN
3,0 days 01:30:37,0.469959,-0.147274,-0.870844,NaN,NaN
4,0 days 01:30:38,0.469910,-0.147196,-0.870884,NaN,NaN
...,...,...,...,...,...,...
4988,0 days 02:53:42,-0.155810,-0.987706,-0.010401,79.0,5.0
4989,0 days 02:53:43,-0.156098,-0.987573,-0.010050,79.0,5.0
4990,0 days 02:53:44,-0.156004,-0.987456,-0.011153,79.0,5.0
4991,0 days 02:53:45,-0.155873,-0.987352,-0.011609,79.0,5.0


In [104]:
df = []
df = pd.merge_asof(acc_df, heartrate_df, on='timedelta')
df = pd.merge_asof(df, sleeplabel_df, on='timedelta')

In [105]:
df.head()

,timedelta,accX,accY,accZ,heartrate,sleep
0,0 days 00:01:40,-0.376241,0.925141,-0.008848,83.0,NaN
1,0 days 00:01:41,-0.377814,0.925636,0.003382,83.0,NaN
2,0 days 00:01:42,-0.379298,0.924588,0.003643,83.0,NaN
3,0 days 00:01:43,-0.379019,0.925039,0.000995,83.0,NaN
4,0 days 00:01:44,-0.380270,0.924537,-0.000302,83.0,NaN


In [106]:
df.isnull().sum()

timedelta     0
accX          0
accY          0
accZ          0
heartrate     0
sleep        20
dtype: int64

# fill na sleep data wth 0

In [107]:
df['sleep'] = df["sleep"].fillna(value=0)
df['heartrate'] = df["heartrate"].fillna(df['heartrate'].median())

df2['sleep'] = df2["sleep"].fillna(value=0)
df2['heartrate'] = df2["heartrate"].fillna(df2['heartrate'].median())

# Standardized data

In [108]:
# Standardized data
feature_columns = ['accX', 'accY', 'accZ', 'heartrate']
label_columns = ['sleep']
df_feature = df[feature_columns]
df_label = df[label_columns]


# PREPARE DATA

In [109]:
df_feature.head()

,accX,accY,accZ,heartrate
0,-0.376241,0.925141,-0.008848,83.0
1,-0.377814,0.925636,0.003382,83.0
2,-0.379298,0.924588,0.003643,83.0
3,-0.379019,0.925039,0.000995,83.0
4,-0.380270,0.924537,-0.000302,83.0


In [110]:
df_label.tail()

,sleep
4896,5.0
4897,5.0
4898,5.0
4899,5.0
4900,5.0


# Simple Moving Average (SMA) and Create 2D feature

In [111]:
columns=['accX', 'accY', 'accZ', 'heartrate']

In [112]:
df_feature

,accX,accY,accZ,heartrate
0,-0.376241,0.925141,-0.008848,83.0
1,-0.377814,0.925636,0.003382,83.0
2,-0.379298,0.924588,0.003643,83.0
3,-0.379019,0.925039,0.000995,83.0
4,-0.380270,0.924537,-0.000302,83.0
...,...,...,...,...
4896,0.475862,-0.153555,-0.866447,99.0
4897,0.482310,-0.151454,-0.863023,99.0
4898,0.484474,-0.150245,-0.861839,99.0
4899,0.477683,-0.152877,-0.865124,99.0


In [113]:
df_feature.iloc[:,0]

0      -0.376241
1      -0.377814
2      -0.379298
3      -0.379019
4      -0.380270
          ...   
4896    0.475862
4897    0.482310
4898    0.484474
4899    0.477683
4900    0.476309
Name: accX, Length: 4901, dtype: float64

In [114]:
df_feature_SMA = pd.DataFrame(columns=columns)

In [115]:
df_feature_SMA['accX'] = df_feature.iloc[:,0].rolling(10, min_periods=1).mean()
df_feature_SMA['accY'] = df_feature.iloc[:,1].rolling(10, min_periods=1).mean()
df_feature_SMA['accZ'] = df_feature.iloc[:,2].rolling(10, min_periods=1).mean()
df_feature_SMA['heartrate'] = df_feature.iloc[:,3].rolling(5, min_periods=1).mean()

In [116]:
import plotly.express as px
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1)
for i in range(len(feature_columns)) :
    fig.add_trace(
        go.Scatter(
            name=feature_columns[i],
            x=df_feature.index,
            y=df_feature[feature_columns[i]],
            showlegend=True),
         row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            name=feature_columns[i]+"SMA",
            x=df_feature_SMA.index,
            y=df_feature_SMA[feature_columns[i]],
            showlegend=True),
         row=1, col=1
    )

fig.update_layout(height=800, width=1000, title_text="Comparison between Raw DF and SMA DF")

fig.show()

In [117]:
# ------------ 1D to 2D feature-------------------------------
 # set sliding window parameter

sliding_window= 100
stride_step = 5
n_feature = 4
feature2D_val =[]
target_ = []
df_label_new=[]
df_feature2D = []
df_feature2D_val = []
df_feature2DT_val =[]
df_feature2D_T = []
for t in range( 0 , len(df_feature) -sliding_window , stride_step):
    F2d = np.array(df_feature.iloc[t:t + sliding_window])
    
    df_feature2D_val.append(F2d)
    df_feature2D = np.array(df_feature2D_val)
    
    F2d_T = np.transpose(F2d)
    
    df_feature2DT_val.append(F2d_T)
    df_feature2D_T = np.array(df_feature2DT_val)

    Labels,count = stats.mode(df_label.iloc[t:t+sliding_window])
    df_label_new.append(int(Labels[0][0]))
    target = np.array(df_label_new)

In [125]:
targets = pd.DataFrame(target ,columns=["sleep"])

In [126]:
targets = targets.replace(-1 , 0 ,inplace=False)

In [127]:
# One hot ของ Y ให้เท่ากับ 4 

In [128]:
targets_dummies = pd.get_dummies(targets["sleep"])

In [130]:
targets_dummies

,0,1,2,3,5
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
956,0,0,0,0,1
957,0,0,0,0,1
958,0,0,0,0,1
959,0,0,0,0,1


# Train-Test split (2D Normal)

In [131]:
x_train, x_test, y_train, y_test = train_test_split(df_feature2D , targets_dummies)

# Train-Test split (2D Tranpose)

In [133]:
xT_train, xT_test, yT_train, yT_test = train_test_split(df_feature2D_T , targets_dummies)

# 10.2 LSTM Model Training and Testing

In [138]:
from tensorflow import keras

ModuleNotFoundError: No module named 'tensorflow'

In [136]:
# ------------ LSTM Architecture parameter -------------------------------
# Nlayer (LSTM, dense), Nnode, Activation
LSTM_L1 = 100 # try 200, 300, 400, 500, 1000
LSTM_L2 = 50 # try 50, 100, 150, 200, 250, 300
dropRate_L1 = 0.25
dropRate_L2 = 0.5
D_out = 5
Activation = "Softmax"
n_classes = 5
Input_shape = (inRow, inCol)
# try
#Option #1:
inRow = N_features
inCol = Sliding_windows
# Option #2
inRow = Sliding_windows
inCol = Sliding_windows

NameError: name 'inRow' is not defined